In [69]:
import os
from PIL import Image, ImagePalette

currently_handled = {
    0:"Global Palette",
    1:"GIF Animated",
    2:"Hidden Frames"
}

# Ending data sub-block + trailer + should search for 
end_of_format = "003B"
# Hidden frames sublock + trailer + new block
hidden_frames = "003B21f9"

zip_format = "504B0304"


file_path = "../data/raw/hiddenZip.gif"
output_dir = "../data/processed/"

In [70]:
def count_byte_sequence_in_image(img: Image.Image, search_bytes: str):
    with open(img.filename, 'rb') as f:
        img_data = f.read()

    hex_data = img_data.hex().upper()

    count = hex_data.count(search_bytes.upper())
    return count

In [71]:
def hidden_magic_byte(img: Image.Image):
    with open(img.filename, 'rb') as f:
        img_data = f.read()

    hex_data = img_data.hex().upper()

    count = hex_data.count((end_of_format+zip_format).upper())
    
    search_pattern = (end_of_format + zip_format).upper()
    
    start_idx = hex_data.find(search_pattern)
    if start_idx == -1:
        return 0
    
    # Convert the starting index from hexadecimal string index to byte index
    byte_start_idx = start_idx // 2
    
    # Extract data from the start index to the end
    extracted_data = img_data[byte_start_idx+2:]
    
    # Save the extracted data to the output file
    with open(os.path.join(output_dir, "hidden_magic_byte.zip"), 'wb') as output_file:
        output_file.write(extracted_data)
        
    return count

In [72]:
def get_image_info(img:Image.Image):
    can_fix=[]
    
    metadata = {
        "format": img.format,
        "size": img.size,
        "mode": img.mode,
        "n_frames": getattr(img, "n_frames", 1),
        "is_animated": getattr(img, "is_animated", False),
        "has_global_palette": bool(img.info.get("palette")),
        "sub_block_trailer": count_byte_sequence_in_image(img, end_of_format),
        "hidden_gif_frames": count_byte_sequence_in_image(img, hidden_frames),
        "hidden_magic_byte": hidden_magic_byte(img),
    }
    
    print("GIF Metadata:")
    for key, value in metadata.items():
        print(f"  {key.replace('_', ' ').title()}: {value}")

    # Identify issues
    if not metadata["is_animated"]:
        print("This GIF is not animated.")
        can_fix.append(1)
    else:
        print("This GIF is animated.")

    if not metadata["has_global_palette"]:
        print("Global Palette: Not Present")
        can_fix.append(0)
    else:
        print("Global Palette: Present")
        
    if metadata["hidden_gif_frames"] != 0:
        can_fix.append(2)
    else:
        print("No hidden gif frames detected")
    
    return {
        "metadata": metadata,
        "fixable_issues": can_fix
    }

In [73]:
def fix_hidden_frames(img: Image.Image, dest: str):
    with open(img.filename, 'rb') as f:
        img_data = f.read()

    hex_data = img_data.hex().upper()

    # Remove trailing 3B
    img_fixed_hex = hex_data.replace(hidden_frames.upper(), "0021f9")
    
    img_fixed_bytes = bytes.fromhex(img_fixed_hex)

    with open(f"{dest}_hidden_frames.gif", 'wb') as f:
        f.write(img_fixed_bytes)

In [74]:
def fix_global_palette(img: Image.Image, dest: str):
    rd_palette = ImagePalette.random()
    bw_palette = [i for i in range(256) for _ in range(3)]
    bw_palette[0:3] = [255, 255, 255]
    bw_palette = ImagePalette.ImagePalette(mode="RGB", palette=bw_palette)
    
    if img.is_animated:
        frames_rd = []
        frames_bw = []
        
        total_frames = img.n_frames
        
        # Iterate through all frames using seek and tell as written in PILLOW doc
        # https://pillow.readthedocs.io/en/latest/handbook/image-file-formats.html#gif
        for frame_index in range(total_frames):
            img.seek(frame_index)
            frame_rd = img.copy()
            frame_bw = img.copy()
            
            if frame_rd.mode != "P":
                frame_rd = frame_rd.convert("P")
                frame_bw = frame_bw.convert("P")

            # Random color palette
            frame_rd.putpalette(rd_palette.palette) 
            frames_rd.append(frame_rd)
            
            # Black color palette
            frame_bw.putpalette(bw_palette.palette)
            frames_bw.append(frame_bw)


        img_rd = frames_rd[0]
        img_rd.save(f"{dest}_animated_random.gif", save_all=True, append_images=frames_rd[1:], loop=0, duration=img.info['duration'])
        
        img_bw = frames_bw[0]
        img_bw.save(f"{dest}_animated_blackwhite.gif", save_all=True, append_images=frames_bw[1:], loop=0, duration=img.info['duration'])
    
    else:
        if img.mode != "P":
            img = img.convert("P")
            
        img.putpalette(rd_palette.palette)
        img.save(f"{dest}_random.gif")
        
        img.putpalette(bw_palette.palette)
        img.save(f"{dest}_blackwhite.gif")

    return img

In [75]:
def analyze_gif_with_pillow(file_path):
    with Image.open(file_path) as img:
        if img.format != "GIF":
            return -1
        
        result = get_image_info(img)
        fixable_issues = result["fixable_issues"]
        
        
        if fixable_issues == None:
            return 0

        for issue in fixable_issues:
            if issue == 0:
                print(f"Fixing: {currently_handled[issue]}")
                fix_global_palette(img, os.path.join(output_dir, f"FIXED_{currently_handled[0]}"))
    
                
            if issue == 1:
                print(f"Fixing: {currently_handled[issue]}")
                #fixed_img = fix_global_palette(img)
                #fixed_img.save(os.path.join(output_dir, f"FIXED_{currently_handled[0]}_{original_name}.gif"))
            
            if issue == 2:
                print(f"Fixing: {currently_handled[issue]}")
                fix_hidden_frames(img, os.path.join(output_dir, f"FIXED_{currently_handled[0]}"))
            

analyze_gif_with_pillow(file_path)


GIF Metadata:
  Format: GIF
  Size: (194, 154)
  Mode: P
  N Frames: 12
  Is Animated: True
  Has Global Palette: False
  Sub Block Trailer: 2
  Hidden Gif Frames: 0
  Hidden Magic Byte: 1
This GIF is animated.
Global Palette: Not Present
No hidden gif frames detected
Fixing: Global Palette
